In [13]:
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

**Import the dataset**

In [15]:
data = pd.read_csv(filepath_or_buffer='./diabetes.csv')

In [24]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Extract X as all columns except the last column and Y as last column**

In [34]:
y = data.Outcome
x = data.drop('Outcome',axis=1)

In [35]:
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [36]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

**Visualize the dataset**

**Split the data into training set and testing set**

In [39]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

NameError: name 'train_test_split' is not defined

 **Logistic regression** 